In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

In [ ]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

  
    plt.show()  

# Loading Data / Making Data


In [ ]:
# Data directory
DATADIR = 'UCI HAR Dataset'
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

1. These are Raw data signals which are given in Inertial Signals form.
2. Signals are taken from sensors (Accelerometer and Gyroscope).
3. The signals are in x,y,z directions that is 3-axial linear acceleration and 3-axial angular velocity.
4. Sensor signals are filtered to have only body acceleration.
5. Triaxial acceleration from the accelerometer is total acceleration.

In [ ]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/UCI HAR Dataset-20221223T170042Z-001/UCI HAR Dataset/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

   
    return np.transpose(signals_data, (1, 2, 0))



1. Transpose is used to change the dimensionality of the output.
2. In this we are aggregating the signals by combination of sample/timestep.
3. Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)

In [ ]:
def load_y(subset):
    filename = f'/content/drive/MyDrive/UCI HAR Dataset-20221223T170042Z-001/UCI HAR Dataset/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()


This function is made to predict the value if integer between 1 to 6 which means the name of activity performed by the employee.

In [ ]:
def load_data():
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

This function is made to obtain the dataset and then returning the X_train, X_test, y_train and y_test.

In [ ]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [ ]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
X_train, X_test, Y_train, Y_test = load_data()

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [ ]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 11s 10ms/step - loss: 1.1753 - accuracy: 0.5068 - val_loss: 0.8943 - val_accuracy: 0.6240
Epoch 2/30
460/460 [==============================] - 4s 9ms/step - loss: 0.7942 - accuracy: 0.6503 - val_loss: 0.8490 - val_accuracy: 0.6590
Epoch 3/30
460/460 [==============================] - 4s 9ms/step - loss: 0.6732 - accuracy: 0.7160 - val_loss: 0.7309 - val_accuracy: 0.7255
Epoch 4/30
460/460 [==============================] - 5s 10ms/step - loss: 0.5171 - accuracy: 0.8158 - val_loss: 0.6020 - val_accuracy: 0.7917
Epoch 5/30
460/460 [==============================] - 5s 11ms/step - loss: 0.3822 - accuracy: 0.8823 - val_loss: 0.4634 - val_accuracy: 0.8544
Epoch 6/30
460/460 [==============================] - 4s 9ms/step - loss: 0.3026 - accuracy: 0.9100 - val_loss: 0.3910 - val_accuracy: 0.8731
Epoch 7/30
460/460 [==============================] - 4s 9ms/step - loss: 0.2676 - accuracy: 0.9181 - val_loss: 0.4763 - val_accuracy: 0.8571
Ep

In [ ]:

(Y_test, model.predict(X_test))


93/93 [==============================] - 1s 3ms/step


(array([[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        ...,
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=uint8),
 array([[2.6553087e-03, 4.1852156e-03, 3.3378179e-04, 7.6191735e-01,
         9.9076980e-01, 2.0216472e-04],
        [2.4742526e-03, 3.3918603e-03, 2.7350226e-04, 7.7424568e-01,
         9.9161947e-01, 1.7700148e-04],
        [2.4058449e-03, 3.1807823e-03, 2.5895791e-04, 7.7479273e-01,
         9.9202687e-01, 1.5247073e-04],
        ...,
        [3.2760251e-01, 9.9999917e-01, 1.5887293e-01, 1.2027987e-02,
         3.3336973e-01, 4.0474705e-02],
        [4.5549673e-01, 9.9999785e-01, 3.0009912e-02, 1.3687733e-02,
         2.2017585e-01, 2.3068514e-02],
        [6.1976779e-01, 9.9999857e-01, 1.8448627e-02, 8.3253076e-03,
         1.4952785e-01, 1.1545376e-02]], dtype=float32))

In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 0s 4ms/step - loss: 0.3577 - accuracy: 0.9067

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.3576847314834595, 0.9066847562789917]


In [ ]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(Y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization (BatchN  (None, 128, 48)          192       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 11s 17ms/step - loss: 1.5782 - accuracy: 0.6666 - val_loss: 1.1719 - val_accuracy: 0.7838
Epoch 2/30
460/460 [==============================] - 7s 15ms/step - loss: 0.7550 - accuracy: 0.8629 - val_loss: 0.5161 - val_accuracy: 0.8904
Epoch 3/30
460/460 [==============================] - 7s 15ms/step - loss: 0.4275 - accuracy: 0.9066 - val_loss: 0.3250 - val_accuracy: 0.9013
Epoch 4/30
460/460 [==============================] - 7s 15ms/step - loss: 0.2421 - accuracy: 0.9313 - val_loss: 0.4269 - val_accuracy: 0.8789
Epoch 5/30
460/460 [==============================] - 7s 16ms/step - loss: 0.2511 - accuracy: 0.9170 - val_loss: 0.2255 - val_accuracy: 0.9128
Epoch 6/30
460/460 [==============================] - 8s 18ms/step - loss: 0.2032 - accuracy: 0.9297 - val_loss: 0.2790 - val_accuracy: 0.9111
Epoch 7/30
460/460 [==============================] - 7s 15ms/step - loss: 0.1933 - accuracy: 0.9304 - val_loss: 0.1768 - val_accuracy: 0.931

In [ ]:
# Confusion Matrix
#new_confusion_matrix(Y_test, model.predict(X_test))
Y_test, model.predict(X_test)

93/93 [==============================] - 1s 6ms/step


(array([[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        ...,
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=uint8),
 array([[0.01685969, 0.01107293, 0.00298568, 0.9843538 , 0.9994666 ,
         0.0071684 ],
        [0.01658969, 0.01075445, 0.00304665, 0.98451185, 0.99948347,
         0.00698212],
        [0.0170277 , 0.01072008, 0.00302641, 0.984527  , 0.9994886 ,
         0.00721045],
        ...,
        [0.3314195 , 0.99999106, 0.15813908, 0.02491959, 0.10609382,
         0.05163733],
        [0.34550476, 0.99999547, 0.21357827, 0.01469629, 0.14028569,
         0.06603802],
        [0.3111793 , 0.99999106, 0.19505553, 0.01823996, 0.1649576 ,
         0.08212633]], dtype=float32))

In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 7ms/step - loss: 0.2533 - accuracy: 0.9264

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.2532747685909271, 0.9263657927513123]
